In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import pandas as pd
import time
import csv

In [44]:
def is_stale(element):
    try:
        element.is_enabled()
        return False
    except StaleElementReferenceException:
        return True

url = 'https://www.visitjeju.net/kr/festival/list?menuId=DOM_000001718007000000&cate1cd=cate0000001360#p1&month='
driver = webdriver.Chrome()
driver.get(url)

with open(r'C:\jupyter_projects\Festival\Crawling\0729\01.csv', 'w', encoding = 'utf-8-sig', newline = '') as f:
    csvWriter = csv.writer(f)
    csvWriter.writerow(['축제명', '월', '주소', '세부지역', '상세내용', '조회수', '연락처', '홈페이지', '이미지'])

    pages = 12

    time.sleep(1)

    for page in range(1, 2):

        if page == 0 or page == 10 or page == 11 or page == 12:
            m = page
        else:
            m = '0' + str(page)
        
        driver.get(f'https://www.visitjeju.net/kr/festival/list?menuId=DOM_000001718007000000&cate1cd=cate0000001360#p1&month={m}')
        print(f'Page {m} loaded')

        time.sleep(3)

        event_lists = driver.find_elements(By.CSS_SELECTOR, '.event_list.clear')
        print('event_list')

        # 해당 코드는 특정 월의 진행 중 축제 또는 종료(예정) 축제를 구분해 크롤링 하고 싶을 때 사용
        # 예를 들어, 8월의 진행 중인 축제를 가져오고자 할 때는 evnet_list = event_lists[0]
        # 8월의 종료(예정) 축제를 가져올 때는 event_list = event_list[1]
        # if event_lists:
        #     event_list = event_lists[1]
        #     lis = event_list.find_elements(By.CSS_SELECTOR, 'li')

        # 기본 코드
        for event_list in event_lists:

            lis = event_list.find_elements(By.CSS_SELECTOR, 'li')

            for index in range(len(lis)):
                while True:
                    try:
                        lis = event_list.find_elements(By.CSS_SELECTOR, 'li')  # 요소를 다시 찾기
                        li = lis[index]
                        li.click()
                        time.sleep(5)

                        print('Event list crawling start')

                        # sub_info - 축제 제목
                        title = '-'
                        try:
                            title = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3').text
                        except NoSuchElementException:
                            print('축제 제목 데이터가 없습니다.')
                            pass

                        # 축제 기간 (월)
                        month = page
                        
                        # sub_info - 축제 주소
                        address = '-'
                        try:
                            address = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont').text
                        except NoSuchElementException:
                            print(title)
                            print('축제 주소 데이터가 없습니다.')
                            pass
                        
                        # sub_info - 축제 주소(상세 위치)
                        location = '-'
                        try:
                            location = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont').text.split(' ')[1]
                        except NoSuchElementException:
                            print(title)
                            print('축제 상세 위치 데이터가 없습니다.')
                            pass

                        # 축제 상세 내용
                        description = '-'
                        try:
                            real = driver.find_element(By.CSS_SELECTOR, 'div.real')
                            ps = real.find_elements(By.TAG_NAME, 'p')
                            description = '\n'.join([p.text for p in ps])
                        except NoSuchElementException:
                            print(title)
                            print('축제 상세 내용(글)이 없습니다.')
                            pass

                        # 조회수
                        hits = 0
                        try:
                            hits = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt').text
                        except NoSuchElementException:
                            print(title)
                            print('축제 조회수 데이터가 없습니다.')
                            pass

                        # 전화번호
                        telephone_number = '-'
                        try:
                            telephone_number = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont').text
                        except NoSuchElementException:
                            print(title)
                            print('축제 전화번호 데이터가 없습니다.')
                            pass

                        # 홈페이지
                        website = '-'
                        try:
                            website_element = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(4) > a')
                            website = website_element.text
                        except NoSuchElementException:
                            print(title)
                            print('축제 홈페이지 주소 데이터가 없습니다.')
                            pass
                        
                        # 축제 상세 내용 - image
                        for i in range(1, 5):
                            try:
                                image_element = driver.find_element(By.CSS_SELECTOR, f'#tab0 > div > div._grid-system-grid_.grid-container > div:nth-child({i}) > div > div > div > div > img')
                                image = image_element.get_attribute("src")
                                if image:
                                    break
                            except NoSuchElementException:
                                print(title)
                                print(f'축제 상세 내용(이미지) {i}가 없습니다.')
                                continue
                        
                        csvWriter.writerow([title, month ,address, location, description, hits, telephone_number, website, image])

                        # print(website)
                        
                        driver.back()
                        time.sleep(3)
                        break
                    
                    except StaleElementReferenceException:
                        print("StaleElementReferenceException 발생")
                        # print(f'월 = {page}, 축제 No = {index + 1}')
                        time.sleep(2)
                        break
                    except NoSuchElementException as e:
                        print('Element not found:', e)
                        break  # 요소를 찾을 수 없으면 루프를 탈출
                    except Exception as e:
                        print('Error occurred:', e)
                        break  # 다른 예외 발생 시 루프를 탈출

driver.quit()

Page 01 loaded
event_list
Event list crawling start
Event list crawling start
Event list crawling start
축제 제목 데이터가 없습니다.
-
축제 주소 데이터가 없습니다.
-
축제 상세 위치 데이터가 없습니다.
-
축제 상세 내용(글)이 없습니다.
-
축제 조회수 데이터가 없습니다.
-
축제 전화번호 데이터가 없습니다.
-
축제 홈페이지 주소 데이터가 없습니다.
-
축제 상세 내용(이미지) 1가 없습니다.
-
축제 상세 내용(이미지) 2가 없습니다.
-
축제 상세 내용(이미지) 3가 없습니다.
-
축제 상세 내용(이미지) 4가 없습니다.
Event list crawling start
Event list crawling start
Error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF623ECEEB2+31554]
	(No symbol) [0x00007FF623E47EE9]
	(No symbol) [0x00007FF623D0872A]
	(No symbol) [0x00007FF623CDD995]
	(No symbol) [0x00007FF623D844D7]
	(No symbol) [0x00007FF623D9C051]
	(No symbol) [0x00007FF623D7CDD3]
	(No symbol) [0x00007FF623D4A33B]
	(No symbol) [0x00007FF623D4AED1]
	GetHandleVerifier [0x00007FF6241D8B2D+3217341]
	GetHandleVerifier [0x00007FF624225AF3+3532675]
	GetHandleVerifier [0x000

### 진행중인 축제 크롤링

In [ ]:
def is_stale(element):
    try:
        element.is_enabled()
        return False
    except StaleElementReferenceException:
        return True


url = 'https://www.visitjeju.net/kr/festival/list?menuId=DOM_000001718007000000&cate1cd=cate0000001360#p1&month='
driver = webdriver.Chrome()
driver.get(url)

with open(r'C:\jupyter_projects\Festival\Crawling\0729\ALL.csv', 'w', encoding = 'utf-8-sig', newline = '') as f:
    csvWriter = csv.writer(f)
    csvWriter.writerow(['축제명', '월', '주소', '세부지역', '상세내용', '조회수', '연락처', '홈페이지', '이미지'])

    pages = 12

    time.sleep(1)

    for page in range(0, 1):

        if page == 0 or page == 10 or page == 11 or page == 12:
            m = page
        else:
            m = '0' + str(page)
        
        driver.get(f'https://www.visitjeju.net/kr/festival/list?menuId=DOM_000001718007000000&cate1cd=cate0000001360#p1&month={m}')
        print(f'Page {m} loaded')

        time.sleep(3)

        event_lists = driver.find_elements(By.CSS_SELECTOR, '.event_list.clear')
        print('event_list')

        if event_lists:
            event_list = event_lists[0]
            lis = event_list.find_elements(By.CSS_SELECTOR, 'li')

            for index in range(len(lis)):
                while True:
                    try:
                        lis = event_list.find_elements(By.CSS_SELECTOR, 'li')  # 요소를 다시 찾기
                        li = lis[index]
                        li.click()
                        time.sleep(5)

                        print('Event list crawling start')

                        # sub_info - 축제 제목
                        title = '-'
                        try:
                            title = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3').text
                        except NoSuchElementException:
                            print('축제 제목 데이터가 없습니다.')
                            pass

                        # 축제 기간 (월)
                        month = page
                        
                        # sub_info - 축제 주소
                        address = '-'
                        try:
                            address = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont').text
                        except NoSuchElementException:
                            print(title)
                            print('축제 주소 데이터가 없습니다.')
                            pass
                        
                        # sub_info - 축제 주소(상세 위치)
                        location = '-'
                        try:
                            location = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont').text.split(' ')[1]
                        except NoSuchElementException:
                            print(title)
                            print('축제 상세 위치 데이터가 없습니다.')
                            pass

                        # 축제 상세 내용
                        description = '-'
                        try:
                            real = driver.find_element(By.CSS_SELECTOR, 'div.real')
                            ps = real.find_elements(By.TAG_NAME, 'p')
                            description = '\n'.join([p.text for p in ps])
                        except NoSuchElementException:
                            print(title)
                            print('축제 상세 내용(글)이 없습니다.')
                            pass

                        # 조회수
                        hits = 0
                        try:
                            hits = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt').text
                        except NoSuchElementException:
                            print(title)
                            print('축제 조회수 데이터가 없습니다.')
                            pass

                        # 전화번호
                        telephone_number = '-'
                        try:
                            telephone_number = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont').text
                        except NoSuchElementException:
                            print(title)
                            print('축제 전화번호 데이터가 없습니다.')
                            pass

                        # 홈페이지
                        website = '-'
                        try:
                            website_element = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(4) > a')
                            website = website_element.text
                        except NoSuchElementException:
                            print(title)
                            print('축제 홈페이지 주소 데이터가 없습니다.')
                            pass
                        
                        # 축제 상세 내용 - image
                        for i in range(1, 5):
                            try:
                                image_element = driver.find_element(By.CSS_SELECTOR, f'#tab0 > div > div._grid-system-grid_.grid-container > div:nth-child({i}) > div > div > div > div > img')
                                image = image_element.get_attribute("src")
                                if image:
                                    break
                            except NoSuchElementException:
                                print(title)
                                print(f'축제 상세 내용(이미지) {i}가 없습니다.')
                                continue
                        
                        csvWriter.writerow([title, month ,address, location, description, hits, telephone_number, website, image])

                        # print(website)
                        
                        driver.back()
                        time.sleep(3)
                        break
                    
                    except StaleElementReferenceException:
                        print("StaleElementReferenceException 발생")
                        # print(f'월 = {page}, 축제 No = {index + 1}')
                        time.sleep(2)
                        break
                    except NoSuchElementException as e:
                        print('Element not found:', e)
                        break  # 요소를 찾을 수 없으면 루프를 탈출
                    except Exception as e:
                        print('Error occurred:', e)
                        break  # 다른 예외 발생 시 루프를 탈출

driver.quit()

### 종료(예정) 축제 크롤링

In [50]:
def is_stale(element):
    try:
        element.is_enabled()
        return False
    except StaleElementReferenceException:
        return True


url = 'https://www.visitjeju.net/kr/festival/list?menuId=DOM_000001718007000000&cate1cd=cate0000001360#p1&month='
driver = webdriver.Chrome()
driver.get(url)

with open(r'C:\jupyter_projects\Festival\Crawling\0729\ALL_end.csv', 'w', encoding = 'utf-8-sig', newline = '') as f:
    csvWriter = csv.writer(f)
    csvWriter.writerow(['축제명', '월', '주소', '세부지역', '상세내용', '조회수', '연락처', '홈페이지', '이미지'])

    pages = 12

    time.sleep(1)

    for page in range(0, 1):

        if page == 0 or page == 10 or page == 11 or page == 12:
            m = page
        else:
            m = '0' + str(page)
        
        driver.get(f'https://www.visitjeju.net/kr/festival/list?menuId=DOM_000001718007000000&cate1cd=cate0000001360#p1&month={m}')
        print(f'Page {m} loaded')

        driver.maximize_window()

        time.sleep(3)

        event_lists = driver.find_elements(By.CSS_SELECTOR, '.event_list.clear')
        print('event_list')

        if event_lists:
            event_list = event_lists[1]
            lis = event_list.find_elements(By.CSS_SELECTOR, 'li')

            for idx in range(len(lis)):
                print(idx)
                stale_cnt = 0
                while True:
                    try:
                        lis = event_list.find_elements(By.CSS_SELECTOR, 'li')  # 요소를 다시 찾기
                        li = lis[idx]
                        li.click()
                        time.sleep(5)

                        print('Event list crawling start')

                        # sub_info - 축제 제목
                        title = '-'
                        try:
                            title = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.sub_info_title > h3').text
                        except NoSuchElementException:
                            print('축제 제목 데이터가 없습니다.')
                            pass

                        print(title)

                        # 축제 기간 (월)
                        month = page
                        
                        # sub_info - 축제 주소
                        address = '-'
                        try:
                            address = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont').text
                        except NoSuchElementException:
                            print(title)
                            print('축제 주소 데이터가 없습니다.')
                            pass
                        
                        # sub_info - 축제 주소(상세 위치)
                        location = '-'
                        try:
                            location = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(2) > p.info_sub_cont').text.split(' ')[1]
                        except NoSuchElementException:
                            print(title)
                            print('축제 상세 위치 데이터가 없습니다.')
                            pass

                        # 축제 상세 내용
                        description = '-'
                        try:
                            real = driver.find_element(By.CSS_SELECTOR, 'div.real')
                            ps = real.find_elements(By.TAG_NAME, 'p')
                            description = '\n'.join([p.text for p in ps])
                        except NoSuchElementException:
                            print(title)
                            print('축제 상세 내용(글)이 없습니다.')
                            pass

                        # 조회수
                        hits = 0
                        try:
                            hits = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.cont_wrap.sub_visual > ul > li:nth-child(6) > p.appraisal_cnt').text
                        except NoSuchElementException:
                            print(title)
                            print('축제 조회수 데이터가 없습니다.')
                            pass

                        # 전화번호
                        telephone_number = '-'
                        try:
                            telephone_number = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(3) > p.info_sub_cont').text
                        except NoSuchElementException:
                            print(title)
                            print('축제 전화번호 데이터가 없습니다.')
                            pass

                        # 홈페이지
                        website = '-'
                        try:
                            website_element = driver.find_element(By.CSS_SELECTOR, '#content > div.cont.detail_page.detail_style > div.sub_visual_wrap > div.inner_wrap > div.sub_info_area > div.basic_information > div:nth-child(4) > a')
                            website = website_element.text
                        except NoSuchElementException:
                            print(title)
                            print('축제 홈페이지 주소 데이터가 없습니다.')
                            pass
                        
                        # 축제 상세 내용 - image
                        image = '-'
                        for i in range(1, 5):
                            try:
                                image_element = driver.find_element(By.CSS_SELECTOR, f'#tab0 > div > div._grid-system-grid_.grid-container > div:nth-child({i}) > div > div > div > div > img')
                                image = image_element.get_attribute("src")
                                if image:
                                    break
                            except NoSuchElementException:
                                print(title)
                                print(f'축제 상세 내용(이미지) {i}가 없습니다.')
                                continue
                        
                        csvWriter.writerow([title, month ,address, location, description, hits, telephone_number, website, image])
                        
                        driver.back()
                        time.sleep(3)
                        break
                    
                    except StaleElementReferenceException:
                        print("StaleElementReferenceException 발생")
                        stale_cnt += 1
                        if stale_cnt >= 5:
                            print('go to next festival data')
                            driver.back()
                            time.sleep(2)
                            break
                        driver.refresh()
                        time.sleep(2)
                    except NoSuchElementException as e:
                        print('Element not found:', e)
                        break  # 요소를 찾을 수 없으면 루프를 탈출
                    except Exception as e:
                        print('Error occurred:', e)
                        break  # 다른 예외 발생 시 루프를 탈출

driver.quit()

Page 0 loaded
event_list
0
Event list crawling start
빛의 벙커 '세잔,프로방스의빛/이왈종,중도의 섬 제주' 전
1
Event list crawling start
제주왕벚꽃축제
제주왕벚꽃축제
축제 홈페이지 주소 데이터가 없습니다.
2
Event list crawling start
휴애리 동백축제
3
Event list crawling start
2024 삼다공원 야간콘서트
4
Event list crawling start
2024 청수 곶자왈 반딧불이 축제
5
Event list crawling start
제13회 우도소라축제
제13회 우도소라축제
축제 홈페이지 주소 데이터가 없습니다.
6
Event list crawling start
서귀포 유채꽃 국제걷기대회
서귀포 유채꽃 국제걷기대회
축제 홈페이지 주소 데이터가 없습니다.
7
Event list crawling start
보롬왓 축제
보롬왓 축제
축제 홈페이지 주소 데이터가 없습니다.
8
Event list crawling start
휴애리 봄 수국축제
9
Event list crawling start
라마다 프라자 제주호텔 '투어 프로그램'
라마다 프라자 제주호텔 '투어 프로그램'
축제 홈페이지 주소 데이터가 없습니다.
10
Event list crawling start
휴애리 핑크뮬리축제
11
Event list crawling start
제주들불축제
12
Event list crawling start
휴애리 매화축제
13
Event list crawling start
마노르블랑 핑크뮬리축제
14
Event list crawling start
한라산청정고사리축제
15
Event list crawling start
세상에 이런
세상에 이런
축제 상세 내용(이미지) 1가 없습니다.
16
Event list crawling start
2023 Miracle 365 아이스버킷 챌린지 RUN IN JEJU
2023 Miracle 365 아이스버킷 챌린지 RUN IN JE